# Import statements

In [1]:
import os
import pandas
import netCDF4
import numpy as np
import datetime
import xml.etree.ElementTree as ET

from bokeh.io import output_notebook
from bokeh.plotting import show, figure

from imports.import_csv import CsvFile
from imports.import_netCDF import NetCDFFile
from imports.import_xml import XmlFile
from utils import logger, from_bokeh_timestamp, to_bokeh_timestamp
from visualize import create_plot, check_filetype_and_data, fill_between

In [2]:
output_notebook()

Loading BokehJS ...

# CSV reader

In [2]:
fpana = r"d:\konin_de\Documents\Repos_Dave\trunk\Scripts\Python_scripts\Visualize\data\test.csv"

In [3]:
df = pandas.read_csv(fpana, ";")

In [4]:
df.columns = ["date", "val"]

In [5]:
file_ = CsvFile(fpana)
file_.get_data()

In [6]:
file_.df

,date,val
0,2017-04-08 12:00:00,0.20085
1,2017-04-08 12:10:00,0.19488
2,2017-04-08 12:20:00,0.18945
3,2017-04-08 12:30:00,0.18471
4,2017-04-08 12:40:00,0.18095
5,2017-04-08 12:50:00,0.17863
6,2017-04-08 13:00:00,0.17833
7,2017-04-08 13:10:00,0.18063
8,2017-04-08 13:20:00,0.18613
9,2017-04-08 13:30:00,0.19541


In [5]:
file_._get_columns()

In [6]:
file_.columns

Index(['08-04-17 12:00;0.20085'], dtype='object')

# NetCDF reader

In [7]:
with netCDF4.Dataset(r"d:\konin_de\Documents\Repos_Dave\trunk\Scripts\Python_scripts\Visualize\data\test.nc", "r") as nc_file:
    print(nc_file.variables.keys())
    variables_list = list(nc_file.variables.keys())

odict_keys(['time', 'observation_id', 'branchid', 'chainage', 'water_level', 'water_salinity', 'water_discharge'])


In [8]:
fpana = r"d:\konin_de\Documents\Repos_Dave\trunk\Scripts\Python_scripts\Visualize\data\test.nc"
variable = "water_level"

In [9]:
with netCDF4.Dataset(fpana, "r") as nc_file:
    vardata = nc_file.variables[variable][:]
    timeseries = range(vardata.shape[0])
    if "time" in nc_file.variables.keys():
        vartime = nc_file.variables["time"][:]
        units_str = nc_file.variables["time"].units
        time_unit = units_str.split()[0]
        date_ = units_str.split()[2]
        time_ = units_str.split()[3]
        ref_date = datetime.datetime.strptime(" ".join([date_, time_]),
                                              "%Y-%m-%d %H:%M:%S")
        timeseries = [add_timedelta(time_unit, int(timestep), ref_date) for timestep in vartime]

    print(nc_file.variables["time"])

NameError: name 'add_timedelta' is not defined

In [10]:
def add_timedelta(timeunit, timesteps, ref_date):
    input_dict = {timeunit: timesteps}
    new_date = ref_date + datetime.timedelta(**input_dict)
    return new_date

In [11]:
with netCDF4.Dataset(fpana, "r") as nc_file:
    vardata = nc_file.variables["observation_id"][:]

In [12]:
list_ = ["".join(np.array(item, dtype=str)).strip() for item in vardata]

In [13]:
fpana = r"d:\konin_de\Documents\Repos_Dave\trunk\Scripts\Python_scripts\Visualize\data\test.nc"
variable = "water_level"
loc = "Maasmond"
file_ = NetCDFFile(fpana)
file_.get_data(variable)
file_.get_timeseries(loc)

water_level
water_salinity
water_discharge


In [18]:
file_.variables

['time',
 'observation_id',
 'branchid',
 'chainage',
 'water_level',
 'water_salinity',
 'water_discharge']

In [21]:
with netCDF4.Dataset(fpana, "r") as nc_file:
#     print(nc_file.variables)
    print(dir(nc_file.variables["water_level"]))
    print("")
    print(nc_file.variables["water_level"].dimensions)

['_FillValue', '__array__', '__class__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__orthogonal_indexing__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '_assign_vlen', '_cmptype', '_enumtype', '_get', '_getdims', '_getname', '_grp', '_grpid', '_has_lsd', '_iscompound', '_isenum', '_isprimitive', '_isvlen', '_name', '_nunlimdim', '_put', '_toma', '_varid', '_vltype', 'assignValue', 'chunking', 'datatype', 'delncattr', 'dimensions', 'dtype', 'endian', 'filters', 'getValue', 'get_var_chunk_cache', 'getncattr', 'group', 'long_name', 'mask', 'name', 'ncattrs', 'ndim', 'renameAttribute', 'scale', 'set_auto_mask', 'set_auto_maskandscale', 'set_auto_scale', 'set_var_chunk_cache', 'setncattr', 'setncattr_str

In [22]:
new_df = pandas.DataFrame(data={"A": [1,2,3,4], "B": [np.nan, np.nan, np.nan, np.nan], "C": [np.nan, np.nan, 1, 2]})

In [28]:
list(new_df.dropna(how="all", axis=1).columns)

['A', 'C']

# XML reader

In [3]:
tree = ET.parse(xml_fpana)

In [35]:
ns = "{http://www.wldelft.nl/fews/PI}"

In [4]:
tree

In [5]:
tree.findall

<bound method ElementTree.findall of <xml.etree.ElementTree.ElementTree object at 0x0000000004D1B2E8>>

In [13]:
tree.getroot().findall("series")

[]

In [31]:
series_list = tree.getroot().findall("{http://www.wldelft.nl/fews/PI}" + "series")

In [32]:
sel_ser = series_list[0]

In [41]:
header_ser = sel_ser.find(ns + "header")

In [45]:
header_ser.findtext(ns + "parameterId")

'H'

In [54]:
event_list = series_list[0].findall("{http://www.wldelft.nl/fews/PI}" + "event")

In [55]:
event = event_list[0]

In [63]:
date_str = " ".join([event.get("date"), event.get("time")])
val_str = event.get("value")
date_fmt = "%Y-%m-%d %H:%M:%S"

date_ = datetime.datetime.strptime(date_str, date_fmt)

In [64]:
date_

datetime.datetime(2017, 3, 15, 13, 0)

In [53]:
event_list[0].get("time")

'13:00:00'

In [31]:
tree.getroot().text

'\n    '

In [8]:
with open(xml_fpana, "r") as fx:
    lines = fx.readlines()

In [9]:
split_line = lines[1].split()

In [10]:
split_line

['<TimeSeries',
 'xmlns="http://www.wldelft.nl/fews/PI"',
 'xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"',
 'xsi:schemaLocation="http://www.wldelft.nl/fews/PI',
 'http://fews.wldelft.nl/schemas/version1.0/pi-schemas/pi_timeseries.xsd"',
 'version="1.2">']

In [28]:
for item in split_line:
    if "xmlns=" in item:
        DEFAULT_NS = item.split("=")[-1].strip('"')

In [29]:
DEFAULT_NS

'http://www.wldelft.nl/fews/PI'

In [30]:
if DEFAULT_NS:
    print(True)

True


In [24]:
DEFAULT_NS = None

In [25]:
if isinstance(DEFAULT_NS, str):
    print(True)

# Load Multiple Files

In [2]:
# fpath_root = os.path.abspath(os.path.dirname(__file__))
fpath_root = r"d:\konin_de\Documents\Repos_Dave\trunk\Scripts\Python_scripts\Visualize"
fpath_data = os.path.join(fpath_root, "data")
fname_nc = "test.nc"
fname_csv = "test.csv"
fname_xml = "test.xml"

# initialize
fpana_nc = os.path.join(fpath_data, fname_nc)
fpana_csv = os.path.join(fpath_data, fname_csv)
fpana_xml = os.path.join(fpath_data, fname_xml)
files = [fpana_nc, fpana_xml, fpana_csv]

Files_list = []
for fname_path in files:
    file_name = os.path.basename(fname_path)
    Files_list.append(check_filetype_and_data(fname_path))


['Maasmond', 'NW_1033.00', 'MO_1034.00', 'MO_1035.00', 'HKVHLD', 'HOEKVHLRTOVR', 'NW_1030.00', 'NW_1031.00', 'NW_1032.00', 'CK_1030.00', 'CK_1031.00', 'CK_1032.00', 'Beerkanaal_1', 'Europahaven_3', 'Europahaven_2', 'Europahaven_4', 'Mississippihaven_1', 'Beerkanaal_4', 'Beneluxhaven_2', 'Beneluxhaven_1', 'Mississippihaven_4', 'Mississippihaven_3', 'Mississippihaven_2', 'Hartelhaven_1', 'Hartelhaven_2', 'Mississippihaven_3.5', 'Dintelhaven_1', 'Dintelhaven_2', 'Dintelhaven_3', 'Dintelhaven_4', 'Dintelhaven_5', 'SUURHBNZDE', 'HK_17.00', 'HK_18.00', 'HK_19.00', 'HK_20.00', 'Koningshaven', 'ROTTDM', 'NM_1000.00', 'NM_999.00', 'Nieuwe maas Boompjes', 'NM_1001.00', 'Rijnhaven_1', 'Rijnhaven_2', 'NM_1002.00', '3ePetroleumhaven_2', 'CK_1019.00', 'CK_1020.00', 'CK_1021.00', 'CK_1022.00', 'CK_1023.00', 'CK_1024.00', 'CK_1025.00', 'CK_1026.00', 'CK_1027.00', 'CK_1028.00', 'CK_1029.00', 'NI_978.00', 'NI_979.00', 'NrdgatvdVisschen_2', 'NM_1003.00', 'NM_1004.00', 'NM_1005.00', 'NM_1008.00', 'NM_1009

water_level
water_salinity
water_discharge


In [12]:
Files_list[0].df

,date,val
0,2017-01-12 11:00:00,0.360284
1,2017-01-12 11:10:00,0.376794
2,2017-01-12 11:20:00,0.396719
3,2017-01-12 11:30:00,0.422933
4,2017-01-12 11:40:00,0.454873
5,2017-01-12 11:50:00,0.482695
6,2017-01-12 12:00:00,0.503206
7,2017-01-12 12:10:00,0.520248
8,2017-01-12 12:20:00,0.535617
9,2017-01-12 12:30:00,0.549426


In [8]:
p = create_plot(Files_list)

test.nc blue 3
test.xml red 2
test.csv green 1


In [11]:
show(p)

# Test Fill_Between

In [3]:
# fpath_root = os.path.abspath(os.path.dirname(__file__))
fpath_root = r"d:\konin_de\Documents\Repos_Dave\trunk\Scripts\Python_scripts\Visualize"
fpath_data = os.path.join(fpath_root, "data")
fname_csv = "test.csv"
fname_csv2 = "test_fill_between.csv"

# initialize
fpana_csv = os.path.join(fpath_data, fname_csv)
fpana_csv2 = os.path.join(fpath_data, fname_csv2)
files = [fpana_csv, fpana_csv2]

Files_list = []
for fname_path in files:
    file_name = os.path.basename(fname_path)
    Files_list.append(check_filetype_and_data(fname_path))


In [7]:
p2 = create_plot(Files_list)
show(p2)

test.csv blue 3
test_fill_between.csv red 2


In [8]:
p3 = fill_between(Files_list)
show(p3)

test.csv red 1
test_fill_between.csv red 1


In [148]:
def fill_between(files):
    Tools_x = "xwheel_zoom, xpan"

    p = figure(x_axis_type="datetime", tools=Tools_x,
               active_scroll="xwheel_zoom", active_drag="xpan")

    colors = ["red", "red", "green"]
    sizes = [1, 1, 1]

    for file_, color, sz in zip(files, colors, sizes):
        logger.info("%s %s %s", file_.fname, color, sz)
        if file_.plot_type == "line":
            p.line(file_.df["date"], file_.df["val"], color=color,
                   line_width=sz)
        elif file_.plot_type == "point":
            p.circle(file_.df["date"], file_.df["val"], color=color,
                     marker_size=sz)

    if len(files) < 2:
        return p

    if len(files) > 2:
        files = files[:2]
    
    x1 = list(files[0].df["date"].map(to_bokeh_timestamp).values)
    x2 = list(files[1].df["date"].map(to_bokeh_timestamp).values)[::-1]
    y1 = list(files[0].df["val"].values)
    y2 = list(files[1].df["val"].values)[::-1]
    
    xs = x1 + x2
    ys = y1 + y2

    p.patches([xs], [ys], alpha=0.3)
    return p

In [4]:
file1 = Files_list[0]
file2 = Files_list[1]

In [5]:
x1 = list(file1.df["date"].map(to_bokeh_timestamp).values)
x2 = list(file2.df["date"].map(to_bokeh_timestamp).values)[::-1]
y1 = list(file1.df["val"].values)
y2 = list(file2.df["val"].values)[::-1]

In [ ]:
def check_same_length(x1, x2):
    return len(x1) == len(x2)

def check_same_values(x1, x2):
    if len(set(x1) - set(x2)) == 0 and len(set(x2) - set(x1)) == 0):
        return True
    else:
        return False

def determine_index_first(x, val):
    lst1 = [i for dt, i in enumerate(x) if dt <= val]
    if lst1:
        s1 = lst1[-1] + 1
    else:
        None
    return s1

def determine_index_last(x, val):
    lst1 = [i for dt, i in enumerate(x) if dt <= val]
    if lst1:
        s1 = lst1[-1]
    else:
        s1 = None
    return s1
    
def clip_series(x1, x2):
    first_val = np.max([x1[0], x2[0]])
    if first_val == x1[0]:
        s1 = determine_index_first(x2, first_val)
        x1_clip1 = x1[:]
        x2_clip1 = x2[s1:]
    else:
        s1 = determine_index_first(x1, first_val)
        x1_clip1 = x1[s1:]
        x2_clip1 = x2[:]

    last_val = np.min([x1[-1], x2[-1]])
    if last_val == x1[-1]:
        s2 = determine_index_last(x2, last_val)
        x1_clip2 = x1_clip1[:]
        x2_clip2 = x2_clip1[:s2]
    else:
        s2 = determine_index_last(x1, last_val)
        x1_clip2 = x1_clip1[:s2]
        x2_clip2 = x2_clip1[:]
    
    return x1_clip2, x2_clip2
    
def interpolate(x1, x2, y1, y2, x_mid):
    y_mid = (x_mid - x1) * (y2 - y1) / (x2 - x1) + y1
    return y_mid
    
def add_dummy_interpolation(x1, x2, y1, y2):
    first_val = np.max([x1[0], x2[0]])
    if first_val == x1[0]:
        s1 = determine_index_first(x2, first_val)
        x1_clip1 = x1[:]
        x2_clip1 = x2[s1:]
        if x1_clip1[0] == x2_clip1[0]:
            pass
        else:
            x2_clip1 = [interpolate(x2[s1 - 1], x2[s1], y2[s1 - 1], y2[s1], x1[0])] + x2_clip1
    else:
        s1 = determine_index_first(x1, first_val)
        x1_clip1 = x1[s1:]
        x2_clip1 = x2[:]
        if x1_clip1[0] == x2_clip1[0]:
            pass
        else:
            x1_clip1 = [interpolate(x1[s1 - 1], x1[s1], y1[s1 - 1], y1[s1], x2[0])] + x1_clip1

    last_val = np.min([x1[-1], x2[-1]])
    if last_val == x1[-1]:
        s2 = determine_index_last(x2, last_val)
        x1_clip2 = x1_clip1[:]
        x2_clip2 = x2_clip1[:s2]
        if x1_clip2[-1] == x2_clip2[-1]:
            pass
        else:
            x2_clip2 = x2_clip2 + [interpolate(x2[s2], x2[s2 + 1], y2[s2], y2[s2 + 1], x1[-1])]
    else:
        s2 = determine_index_last(x1, last_val)
        x1_clip2 = x1_clip1[:s2]
        x2_clip2 = x2_clip1[:]
        if x1_clip2[-1] == x2_clip2[-1]:
            pass
        else:
            x1_clip2 = x1_clip2 + [interpolate(x1[s2], x1[s2 + 1], y1[s2], y1[s2 + 1], x2[-1])]
    
    return x1_clip2, x2_clip2
    

In [ ]:
check_same_length()
check_same_values()
clip_series()
add_dummy_interpolation()

In [126]:
new_1 = x1 + x2
new_2 = y1 + y2

In [129]:
p = figure()
p.patches([new_1],[new_2])
show(p)

In [137]:
to_bokeh_timestamp(from_bokeh_timestamp(730101212300, time_unit="milliseconds"), time_unit="milliseconds")

730101212300.0

In [149]:
p = fill_between(Files_list)
show(p)

test.csv red 1
test_fill_between.csv red 1
